In [8]:
from subprocess import call
from multiprocessing import Pool
import pandas as pd
import numpy as np

In [6]:
def vcf2plink(in_vcf, out_prefix, sample_file=None, mac=10):
    '''
    only keep biallelic variants
    remove variants with maf == 0
    extract genotype of samples in sample file
    '''
    if sample_file:
        extract_samples = f' --keep {sample_file} '
    else:
        extract_samples = ''
    cmd = f'''plink --vcf {in_vcf} \
            --make-bed \
            --biallelic-only \
            --out {out_prefix} \
            --mac {mac} \
            {extract_samples} \
            --keep-allele-order > {out_prefix}.log'''

    call(cmd, shell=True)
    call(f'rm {out_prefix}.nosex', shell=True)


In [4]:
chroms = range(1,23)

In [7]:
clean_vcf('../cojo_finemap/ALSPAC740_simp/21.ALSPAC.beagle.anno.csq.shapeit.20131101.vcf.gz', 'UK10K.chr21')

In [10]:
samples = pd.read_csv('/h/jianhua/REF/1000G/20130502/integrated_call_samples_v3.20130502.ALL.panel',sep='\t')
for pop,df in samples.groupby('super_pop'):
    df['sample'].to_csv(f'/h/jianhua/REF/1000G/20130502/samples/{pop}.samples', sep='\t', index=False, header=False)

In [17]:
pairs = []
for pop in ['EUR','EAS','AFR','AMR','SAS']:
    for chrom in range(1,23):
        pairs.append([f'/h/jianhua/REF/1000G/20130502/ALL.chr{chrom}.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz',
          f'/h/jianhua/REF/1000G/20130502/pop_vcf/{pop}.chr{chrom}.vcf.gz', f'/h/jianhua/REF/1000G/20130502/samples/{pop}.samples'])

p = Pool(30)
p.starmap(clean_vcf, pairs)
p.close()

In [1]:
def vcf2plink(in_vcf, plink_prefix):
    call(f'plink --vcf {in_vcf}  --make-bed --out {plink_prefix} --keep-allele-order > {plink_prefix}.log', shell=True)
    # bim = pd.read_csv(f'{plink_prefix}.bim', delim_whitespace=True, header=None)
    # bim[1] = bim[3].astype(str)+'-'+bim[5]+'-'+bim[4]
    # bim.to_csv(f'{plink_prefix}.bim', sep='\t', index=False, header=False)
    call(f'rm {plink_prefix}.log {plink_prefix}.nosex', shell=True)

In [5]:
pairs = []
for pop in ['EUR','EAS','AFR','AMR','SAS']:
    for chrom in range(1,23):
        pairs.append([f'/h/jianhua/REF/1000G/20130502/pop_vcf/{pop}.chr{chrom}.vcf.gz', f'/h/jianhua/REF/1000G/20130502/pop_plink/{pop}.chr{chrom}'])

p = Pool(30)
p.starmap(vcf2plink, pairs)
p.close()

In [17]:
df = pd.read_csv('./CA448.chr1.500.UK10K.jma.cojo', sep='\t')

In [18]:
df

,Chr,SNP,bp,refA,freq,b,se,p,n,freq_geno,bJ,bJ_se,pJ,LD_r
0,1,2151579-C-A,2151579,A,0.24160,-0.0275,0.0067,4.052300e-05,9657.60,0.241567,-0.089524,0.007791,1.467240e-30,0.492685
1,1,2160390-C-G,2160390,G,0.07058,0.0335,0.0111,2.544340e-03,9836.23,0.070576,0.095259,0.012826,1.111940e-13,0.049267
2,1,2231595-A-C,2231595,C,0.06435,0.0264,0.0114,2.056980e-02,10164.10,0.064349,0.253818,0.016213,3.040740e-55,0.107538
3,1,2240006-T-C,2240006,C,0.87000,-0.0412,0.0080,2.604860e-07,10966.60,0.870005,-0.397382,0.014735,3.464950e-160,0.595557
4,1,2245570-C-G,2245570,G,0.74160,-0.0343,0.0065,1.313710e-07,9800.52,0.741567,-0.196135,0.011761,1.955900e-62,0.842868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,1,222911002-G-C,222911002,C,0.49920,-0.0402,0.0059,9.521310e-12,9097.46,0.499222,-0.135271,0.007839,1.004730e-66,-0.001720
176,1,222996633-A-G,222996633,G,0.25970,-0.0232,0.0071,1.084610e-03,8200.07,0.259730,-0.286729,0.017628,1.730570e-59,0.918196
177,1,223004728-C-T,223004728,T,0.22810,0.0225,0.0073,2.054820e-03,8471.46,0.228075,0.321674,0.018276,2.425350e-69,-0.125973
178,1,223213214-C-T,223213214,T,0.10430,0.0291,0.0098,2.983940e-03,8860.21,0.104307,0.083997,0.010561,1.814570e-15,-0.010820


In [19]:
len(df[(df['p']<=5e-8)&(df['pJ']<=5e-8)])

44

In [20]:
df[(df['p']<=5e-8)&(df['pJ']<=5e-8)]

,Chr,SNP,bp,refA,freq,b,se,p,n,freq_geno,bJ,bJ_se,pJ,LD_r
6,1,2252191-A-G,2252191,G,0.19410,0.0395,0.0071,2.645980e-08,10060.30,0.194084,0.695869,0.020106,1.760450e-262,0.848000
7,1,2252205-C-T,2252205,T,0.14370,-0.0469,0.0082,1.068300e-08,9584.76,0.143747,-0.511179,0.017824,6.895960e-181,0.363905
17,1,2990994-C-T,2990994,T,0.38920,-0.0341,0.0061,2.268550e-08,8964.61,0.389206,-0.623707,0.018175,4.377660e-258,0.010336
28,1,3325912-C-A,3325912,A,0.13960,-0.0514,0.0084,9.413630e-10,9351.74,0.139595,-0.433271,0.018199,2.823280e-125,-0.166863
39,1,38408974-T-C,38408974,C,0.44550,0.0353,0.0058,1.156170e-09,9538.56,0.445511,0.511350,0.013806,2.655000e-300,-0.576618
42,1,38456106-G-A,38456106,A,0.40690,0.0326,0.0052,3.628360e-10,12154.50,0.406850,0.208449,0.009237,9.099240e-113,0.464664
47,1,55487648-A-G,55487648,G,0.20710,-0.0506,0.0074,8.039730e-12,8802.63,0.207058,-0.392950,0.014464,1.535650e-162,-0.196181
58,1,55505647-G-T,55505647,T,0.02206,0.2406,0.0247,2.017780e-22,5951.37,0.022055,0.351065,0.028950,7.635470e-34,-0.274917
70,1,56890205-C-G,56890205,G,0.27040,0.0360,0.0066,4.909830e-08,9229.88,0.270368,0.147193,0.009722,8.707820e-52,0.662901
72,1,56932462-C-T,56932462,T,0.78980,-0.0491,0.0071,4.662390e-12,9460.53,0.789829,-0.534612,0.012770,0.000000e+00,-0.633726


In [13]:
df = pd.read_csv('./CA448.chr1.500.UK10K.extract500k.jma.cojo', sep='\t')

In [14]:
df

,Chr,SNP,bp,refA,freq,b,se,p,n,freq_geno,bJ,bJ_se,pJ,LD_r
0,1,2151579-C-A,2151579,A,0.24160,-0.0275,0.0067,4.052300e-05,9657.60,0.241567,-0.089524,0.007791,1.467240e-30,0.492685
1,1,2160390-C-G,2160390,G,0.07058,0.0335,0.0111,2.544340e-03,9836.23,0.070576,0.095259,0.012826,1.111940e-13,0.049267
2,1,2231595-A-C,2231595,C,0.06435,0.0264,0.0114,2.056980e-02,10164.10,0.064349,0.253818,0.016213,3.040740e-55,0.107538
3,1,2240006-T-C,2240006,C,0.87000,-0.0412,0.0080,2.604860e-07,10966.60,0.870005,-0.397382,0.014735,3.464950e-160,0.595557
4,1,2245570-C-G,2245570,G,0.74160,-0.0343,0.0065,1.313710e-07,9800.52,0.741567,-0.196135,0.011761,1.955900e-62,0.842868
5,1,2249236-G-A,2249236,A,0.80180,0.0360,0.0070,2.705910e-07,10192.40,0.801764,0.751153,0.020627,2.411380e-290,-0.890322
6,1,2252191-A-G,2252191,G,0.19410,0.0395,0.0071,2.645980e-08,10060.30,0.194084,0.695869,0.020106,1.760450e-262,0.848000
7,1,2252205-C-T,2252205,T,0.14370,-0.0469,0.0082,1.068300e-08,9584.76,0.143747,-0.511179,0.017824,6.895960e-181,0.363905
8,1,2253327-C-T,2253327,T,0.02180,-0.0581,0.0179,1.171170e-03,11635.40,0.021795,-0.172651,0.019654,1.572170e-18,-0.047402
9,1,2255633-G-A,2255633,A,0.03477,-0.0106,0.0131,4.184230e-01,13815.30,0.034769,-0.120458,0.015729,1.884420e-14,0.000000


In [15]:
c = pd.read_csv('./CA448.chr1.500.UK10K.extractsnps.jma.cojo',sep='\t')

In [16]:
c

,Chr,SNP,bp,refA,freq,b,se,p,n,freq_geno,bJ,bJ_se,pJ,LD_r
0,1,2252191-A-G,2252191,G,0.1941,0.0395,0.0071,2.645980e-08,10060.30,0.194084,0.232167,0.012671,5.513490e-75,0.848000
1,1,2252205-C-T,2252205,T,0.1437,-0.0469,0.0082,1.068300e-08,9584.76,0.143747,-0.268878,0.014637,2.297480e-75,0.000000
2,1,3325912-C-A,3325912,A,0.1396,-0.0514,0.0084,9.413630e-10,9351.74,0.139595,-0.051400,0.008416,1.014200e-09,0.000000
3,1,38405257-T-C,38405257,C,0.5846,0.0321,0.0059,5.307820e-08,9383.72,0.584587,0.167615,0.011413,7.942370e-49,0.757800
4,1,38408974-T-C,38408974,C,0.4455,0.0353,0.0058,1.156170e-09,9538.56,0.445511,0.102016,0.009451,3.676580e-27,0.786753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,1,222829550-G-A,222829550,A,0.7289,-0.0699,0.0064,9.060570e-28,9712.06,0.728853,-1.503250,0.018973,0.000000e+00,-0.624816
69,1,222836733-T-C,222836733,C,0.1220,-0.0702,0.0089,3.079530e-15,9316.39,0.121951,-1.277820,0.022371,0.000000e+00,-0.556308
70,1,222899427-T-C,222899427,C,0.6923,0.0581,0.0062,7.187050e-21,9629.93,0.692268,1.141770,0.016543,0.000000e+00,0.659588
71,1,222911002-G-C,222911002,C,0.4992,-0.0402,0.0059,9.521310e-12,9097.46,0.499222,-0.556545,0.010749,0.000000e+00,0.418095
